# Dog salon Promotion's week from 6/16/19 to 6/22/19
**Challenge :** Help the dog salon owner to answer follow these questions
1. To view all the data, we would like to join three tables together using ".pet_id" as the primary and foreign key
2. What size of dogs are visited more than 10 times?
3. How many pet owners come to the salon to receive services?
4. Which dog owner is the most frequent visitor to the shop?
5. What is the most busy day?
6. Which dog and its owner are the most frequent visitors to the salon for services?
7. Who would come for haircut service or get service on June 17?
8. The salon have a promotion to give a gift to those pets who have had their nails done at the salon
9. The town where the dog salon is located had a street fair on June 17 and 18. The business owner wants to know how many transactions took place on those days
10. What size dogs are there for service? and how many of each size?

transactions
       _transaction_id (integer)
        date (date)
        pet_id (integer)
        service (string)_
owners
        _owner_id (integer)
        pet_id (integer)
        size (string), size of pet_
owners_2
        _owner_id (integer)
        pet_id (integer)
        size (string), size of pet_
visits
        _pet_id (integer)
        visits_count (integer)_

**1. To view all the data, we would like to join three tables together using ".pet_id" as the primary and foreign key.**

In [ ]:
SELECT owner_id, tr.pet_id, transaction_id, visits_count 
FROM owners AS own
    LEFT OUTER JOIN transactions AS tr
    ON own.pet_id = tr.pet_id
    LEFT OUTER JOIN visits AS vi
    ON own.pet_id = vi.pet_id
WHERE transaction_id IS NOT NULL
ORDER BY transaction_id;

Error: Catalog Error: Table with name owners does not exist!
Did you mean "temp.information_schema.tables"?

**2.What size of dogs are visited more than 10 times?**

**ANSWER :** There were only four dogs that visited the salon more than 10 times, consisting of three large dogs and one small dog. The dog with the highest number of visits was pet number 23, a large-sized dog that visited the salon 15 times.

In [15]:
SELECT ow.pet_id, size, visits_count AS num_visits
FROM owners.csv AS ow JOIN visits.csv AS v
      ON ow.pet_id = v.pet_id
WHERE visits_count >= 10
ORDER BY num_visits DESC
LIMIT 10;

**3.How many pet owners come to the salon to receive services?**
We have to different teams collected the pet's owner informations
Let's begin by looking at all pet's owner and what service did they have by combine two tables
by using UNION

**ANSWER :** 12 pet owners visited the salon during promotion week!

In [12]:
WITH total_owners AS
(SELECT * FROM owners.csv
UNION
SELECT * FROM owners_2.csv
ORDER BY owner_id)

SELECT COUNT(DISTINCT owner_id)
FROM total_owners;

,count(DISTINCT owner_id)
0,12


**4.Which dog owner is the most frequent visitor to the shop?**

ANSWER : The pet owner no 13 visited the salon 17 times!

In [13]:
WITH new_owner_id AS
 (
    SELECT * FROM owners.csv
    UNION
    SELECT * FROM owners_2.csv
 )
 
SELECT owner_id, SUM(visits_count) AS num_visits
FROM new_owner_id AS own
      JOIN visits.csv AS vi 
      ON own.pet_id = vi.pet_id
GROUP BY owner_id
ORDER BY num_visits DESC;

,owner_id,num_visits
0,13,17.0
1,65,15.0
2,19,11.0
3,99,11.0
4,11,10.0
5,87,8.0
6,34,6.0
7,52,5.0


**5. What is the most busy day?**

**ANSWER :** The busiest day is June 18. There was 4 transactions

In [14]:
WITH new_owner_id AS
 (
    SELECT * FROM owners.csv
    UNION
    SELECT * FROM owners_2.csv
 )

SELECT date, COUNT(date)
FROM transactions.csv AS tr
      JOIN new_owner_id
      ON tr.pet_id = new_owner_id.pet_id
WHERE transaction_id IS NOT NULL
GROUP BY date
ORDER BY COUNT(date) DESC;


,date,count(date)
0,2019-06-18,4
1,2019-06-22,3
2,2019-06-17,3
3,2019-06-20,2
4,2019-06-21,2
5,2019-06-16,2
6,2019-06-19,2


**6.Which dog and its owner are the most frequent visitors to the salon for services?**

**ANSWER :** The most visited is 15 times from the owner id no. 65 with their pet no. 23

In [15]:
WITH new_owners AS
(
  SELECT * FROM owners.csv
  UNION
  SELECT * FROM owners_2.csv
)

SELECT CONCAT(new_owners.owner_id, ', ', new_owners.pet_id) AS owner_pet, visits_count
FROM new_owners JOIN visits.csv AS vi
    ON new_owners.pet_id = vi.pet_id
WHERE visits_count >= 3
ORDER BY visits_count DESC, owner_pet;

,owner_pet,visits_count
0,"65, 23",15
1,"13, 87",13
2,"19, 32",11
3,"99, 32",11
4,"11, 67",10
5,"87, 79",8
6,"34, 13",6
7,"52, 43",4


**7. Who would come for haircut service or get service on June 17?**

In [16]:
SELECT owner_id, tr.pet_id, date, service
FROM transactions.csv AS tr JOIN owners.csv AS own
      ON tr.pet_id = own.pet_id
WHERE date = '2019-06-17'
      OR
      service = 'haircut'
ORDER BY date;

,owner_id,pet_id,date,service
0,11,67,2019-06-16,haircut
1,13,86,2019-06-17,nails
2,34,13,2019-06-17,color
3,87,79,2019-06-17,nails
4,11,67,2019-06-18,haircut
5,87,79,2019-06-18,haircut
6,65,23,2019-06-19,haircut
7,52,43,2019-06-22,haircut


**8.The salon have a promotion to give a gift to those pets who have had their nails done at the salon.**

**ANSWER :** only 5 dogs will get the gift from get the nail service.

In [17]:
SELECT pet_id, service,
CASE
  WHEN service = 'nails' THEN 'gift'
  ELSE 'no gift'
END AS get_gift
FROM transactions.csv
ORDER BY get_gift;

,pet_id,service,get_gift
0,86,nails,gift
1,79,nails,gift
2,86,nails,gift
3,43,nails,gift
4,23,nails,gift
5,67,shampoo,no gift
6,43,haircut,no gift
7,32,shampoo,no gift
8,67,haircut,no gift
9,13,color,no gift


**9.The town where the dog salon is located had a street fair on June 17 and 18. The business owner wants to know how many transactions took place on those days.**

**ANSWER :** On June 17 was 3 transactions and June 18 was 4 transactions.

In [18]:
SELECT date, COUNT(transaction_id)
FROM transactions.csv
WHERE date BETWEEN '2019-06-17' AND '2019-06-18'
GROUP BY date;

,date,count(transaction_id)
0,2019-06-17,3
1,2019-06-18,4


**10.What size dogs are there for service? and how many of each size?**

**ANSWER :** The total number of dogs that received services is 17. There were 7 small size dogs, 4 medium size dogs, and 6 large size dogs.

In [19]:
WITH all_owners AS
(
  SELECT * FROM owners.csv
  UNION
  SELECT * FROM owners_2.csv
)

SELECT size, COUNT(size) AS size_count
FROM all_owners
GROUP BY size
ORDER BY size DESC;

,size,size_count
0,small,7
1,medium,4
2,large,6
